# Patrol Districting with Unequal Workload Constraints

This is an example of creating Patrol Districts using a p-median model, with workload inequality constraints. The solution makes the total number of calls in the resulting districts approximately equal.

In [ ]:
from crimepy import data, pmed
from crimepy.cdcplot import plt

carrollton = data.load_carrollton_data()
carrollton.head()

In [ ]:
# On my machine, Gurobi takes about 2 minutes
# and HiGHS around 30 minutes
# will not solve at all with CBC
if 'GUROBI_CMD' in pmed.av_solv:
    loc_solv = pmed.pulp.GUROBI_CMD
elif 'HiGHS' in pmed.av_solv:
    loc_solv = pmed.pulp.HiGHS
else:
    print('You will need to figure out what solver you want to use')
    loc_solv = None

# Creating the base object
pmed12 = pmed.pmed(carrollton,'CountCalls','PDGrid',
                   ta=12,ine=0.1,th=5260,buffer_distance=2000)

In [ ]:
# 12 area solution
pmed12.solve(solver=loc_solv())
pmed12.map_plot()

In [ ]:
# Can see that there are some isolated components
# can run subtours and then warm start (if your solver allows it)
stres = pmed12.collect_subtours()
pmed12.solve(solver=loc_solv(warmStart=True))
pmed12.map_plot()